<a href="https://colab.research.google.com/github/haarithahmdkhan/HousePrices-project1-/blob/main/final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install SpeechRecognition



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 12.3 MB/s eta 0:00:00


In [2]:
!pip install gtts

In [3]:
!pip install youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 6.8 MB/s eta 0:00:00


In [4]:
!sudo apt-get install portaudio19-dev
!pip install PyAudio==0.2.14

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (1,033 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Front

In [5]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
!pip install pydub

In [ ]:
/content/week2.json/week2.json

In [17]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
import pandas as pd
from datasets import Dataset
import speech_recognition as sr
from pydub import AudioSegment
from gtts import gTTS
import os
from youtube_transcript_api import YouTubeTranscriptApi
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import files

# Load JSON file
file_path = "/content/sample_data/week2.json"
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Create a dataset from the JSON data
def create_dataset(data):
    contexts = []
    questions = []
    answers = []

    for item in data['data']:
        for paragraph in item['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]  # Assuming one answer per question
                contexts.append(context)
                questions.append(question)
                answers.append({"text": [answer['text']], "answer_start": [answer['answer_start']]})

    df = pd.DataFrame({"context": contexts, "question": questions, "answers": answers})
    return Dataset.from_pandas(df)

# Load tokenizer and preprocess dataset
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    answers = examples["answers"]
    inputs["start_positions"] = [
        ans["answer_start"][0] for ans in answers
    ]
    inputs["end_positions"] = [
        ans["answer_start"][0] + len(ans["text"][0]) for ans in answers
    ]
    return inputs

# Create and preprocess dataset
dataset = create_dataset(data)
tokenized_dataset = dataset.map(preprocess, batched=True)

# Split dataset into train and evaluation sets
train_indices, eval_indices = train_test_split(
    range(len(tokenized_dataset)), test_size=0.2, random_state=42
)
train_dataset = tokenized_dataset.select(train_indices)
eval_dataset = tokenized_dataset.select(eval_indices)

# Load model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=500,
    eval_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=2
)

# Define compute metrics function
def compute_metrics(pred):
    from sklearn.metrics import accuracy_score
    predictions = np.argmax(pred.predictions, axis=2)
    true_labels = pred.label_ids
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    return {
        "accuracy": accuracy_score(true_labels.flatten(), predictions.flatten())
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("/content/sample_data/results")

# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

# Save the fine-tuned model and tokenizer
model.save_pretrained("/content/sample_data/fine_tuned_model")
tokenizer.save_pretrained("/content/sample_data/fine_tuned_model")

# Function to process uploaded audio file
def speech_to_text_from_file(file_path):
    recognizer = sr.Recognizer()
    try:
        # Load the audio file
        with sr.AudioFile(file_path) as source:
            print("Processing audio file...")
            audio = recognizer.record(source)  # Record the entire audio file

              # Convert .m4a to .wav if needed
            if file_path.endswith(".m4a"):
              sound = AudioSegment.from_file(file_path, "m4a")
              sound.export("temp.wav", format="wav")  # Convert to .wav and save as temp.wav
              file_path = "temp.wav"  # Update file_path to the converted file
    except Exception as e:
        print(f"Error loading audio file: {e}")
        return None

    try:
        # Recognize speech using Google Web Speech API
        question = recognizer.recognize_google(audio)
        print(f"Extracted question from audio: {question}")
        return question
    except sr.UnknownValueError:
        print("Sorry, the audio was not clear enough to extract text.")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

# Text-to-speech function
def text_to_speech(text):
    tts = gTTS(text)
    tts.save("response.mp3")
    os.system("start response.mp3")

# Answering function
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True)

    # Move inputs to the same device as the model
    device = model.device  # Get the device of the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
    )
    return answer

# Fetch YouTube transcript
def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = " ".join([entry['text'] for entry in transcript])
        return transcript_text
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None

# Main program
if __name__ == "__main__":
    # Upload audio file
    uploaded = files.upload()  # Allows you to select and upload a file
    file_path = list(uploaded.keys())[0]  # Get the uploaded file's name

    video_id = "qqQ8wO-lNmI"  # Replace with actual YouTube video ID
    context = get_transcript(video_id)

    if context:
        print("Transcript fetched successfully.")
        question = speech_to_text_from_file(file_path)
        if question:
            answer = answer_question(question, context)
            print(f"Answer: {answer}")
            text_to_speech(answer)
        else:
            print("No question detected.")
    else:
        print("Could not fetch transcript.")


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-ffa97f510901>:104: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


Evaluation results: {'eval_loss': 5.487765312194824, 'eval_accuracy': 0.75, 'eval_runtime': 3.8042, 'eval_samples_per_second': 0.526, 'eval_steps_per_second': 0.263, 'epoch': 3.0}


Saving Recording_wav.wav to Recording_wav (4).wav
Transcript fetched successfully.
Processing audio file...
Extracted question from audio: what is resistors
Answer: [CLS] what is resistors [SEP] in the last lecture we saw the importance of learning about electronics, method of learning by doing, how it is very useful learning a subject like electronics and a brief history of electronics, about various stages like the vacuum tube, the transistor and integrated circuits. we also saw the plan of the topics to be learnt during this lecture, how to build circuits using breadboard and some ideas about use of digital multimeter and power supply which we would be making use of in doing different experiments in electronics. the plan is to read about three aspects in electronics basically components and devices, the measuring instruments and different circuits. under components and devices, the passive components like resistors, capacitors, active components like transistor and operational ampli

In [8]:
!pip install playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=b176f4b91518e2964f30580c10ff9b084f8b53f4c12aa2f893c65076e2b2a0e3
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [19]:
from playsound import playsound

def text_to_speech(answer):
    tts = gTTS(answer)
    tts.save("response.mp3")
    playsound("response.mp3")  # Use playsound to play the audio
  # Optionally, remove the temporary file